In [1]:
!apt-get install -y git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [2]:
! rm -rf NLP-project
! git clone https://github.com/ReviBa/NLP-project
! cp -rf NLP-project/* .

Cloning into 'NLP-project'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 107 (delta 52), reused 72 (delta 22), pack-reused 0
Receiving objects: 100% (107/107), 6.19 MiB | 8.99 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [3]:
! pip install --q -r /content/requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.7 MB/s eta 

In [4]:
from t5.train import t5_train
from utils.tokens_utils import get_tokenizer_based_on_data, get_max_tokens_number
from context_manager import DialogContextManager
from t5.model import T5Model
import pandas as pd
import warnings
import torch

In [5]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
assert DEVICE == 'cuda'

warnings.filterwarnings('ignore')

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# Using regular T5 was causing problems. he was trained on different languages,
# so sometimes it returned results that are not in english.
MODEL_NAME = "google/t5-v1_1-base"
BATCH_SIZE = 2
PATH = "/content/NLP-project/resources/embeddings_comparison_mini_cosine.csv"
SAVE_TO_CHECKPOINT = "style_transfer_model-10-epochs-on-new-data"

In [9]:
style_transfer_df = pd.read_csv(PATH)
style_transfer_df = style_transfer_df[style_transfer_df['cosine']<=0.45]
cols = ['input', 'target']
train_df =  style_transfer_df.rename(columns={'sentence': cols[0], 'all-MiniLM-L6-v2 with cosine': cols[1]})[cols].drop_duplicates()


In [10]:
train_df.count()

input     27152
target    27152
dtype: int64

In [11]:
tokenizer = get_tokenizer_based_on_data(train_df, *cols)
i_max_token_len, o_max_token_len = get_max_tokens_number(train_df, *cols)

In [12]:
! export CUDA_LAUNCH_BLOCKING=1
! export TORCH_USE_CUDA_DSA=1
trained_model, trainer = t5_train(train_df, DEVICE, tokenizer,
                                  MODEL_NAME, i_max_token_len, o_max_token_len, train_batch_size=BATCH_SIZE,
                                  test_batch_size=BATCH_SIZE, epochs=9, save_to_checkpoint=SAVE_TO_CHECKPOINT, train_from_checkpoint="/content/drive/MyDrive/style_transfer_model-8-epochs-on-new-data.ckpt")

KeyboardInterrupt: ignored

In [ ]:

style_transfer_df = pd.read_csv(PATH)
cols = ['input', 'target']
train_df =  style_transfer_df.rename(columns={'sentence': cols[0], 'all-MiniLM-L6-v2 with cosine': cols[1]})[cols].drop_duplicates()

# List of strings to search for
search_list = ['Scranton', 'Michael Scott', 'Dunder-Mifflin', 'Dunder Mifflin', 'Jim', 'Pam', 'Dwight','Holly', 'Jan']

# Filtering the DataFrame
filtered_df = train_df[train_df['target'].str.contains('|'.join(search_list), na=False)]

In [ ]:
filtered_df

,input,target
67,"Certainly , Mr . Emory . Is there a problem ,...","Dwight, are you talking about us?"
186,"We put $ 500 into the savings account , and $...",Jan called this morning and pledged $500.00.
299,"Hello , Ann ! Is that you ?","Hi, Jan!"
365,You'll be happy to know that it's open 24/7 .,It is Friday morning and it is another beautif...
385,"Say , Jim , how about going for a few beers af...","Jim, good party, huh? Just a little something ..."
...,...,...
69446,John ? I have to say I was bothered by the way...,Am I worried that Jim and David are having a m...
69550,"Ten gallons of premium . That's $ 18.56 , ple...",I need you to get me the prices that you're c...
69575,"Jim and you are bosom friends , aren't you ?",Jim and I are great friends
69663,"Hi , Ben . Come in . Diane's not ready yet , ...","Pam, can you come in here, please? Have a seat"


In [ ]:
trained_model, trainer = t5_train(filtered_df, DEVICE, tokenizer,
                                  MODEL_NAME, i_max_token_len, o_max_token_len, train_batch_size=BATCH_SIZE,
                                  test_batch_size=BATCH_SIZE, epochs=3, save_to_checkpoint="style_transfer_model-9-epochs-on-new-data-and-more", train_from_checkpoint="/content/drive/MyDrive/style_transfer_model-10-epochs-on-new-data.ckpt")

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 254 M 
-----------------------------------------------------
254 M     Trainable params
0         Non-trainable params
254 M     Total params
1,019.717 Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 0, global step 210: val_loss reached 0.35780 (best 0.35780), saving model to "/content/drive/MyDrive/style_transfer_model-9-epochs-on-new-data-and-more.ckpt" as top 2


Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 1, global step 421: val_loss reached 0.30678 (best 0.30678), saving model to "/content/drive/MyDrive/style_transfer_model-9-epochs-on-new-data-and-more-v1.ckpt" as top 2


Validating: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.distributed:Epoch 2, global step 632: val_loss reached 0.29412 (best 0.29412), saving model to "/content/drive/MyDrive/style_transfer_model-9-epochs-on-new-data-and-more.ckpt" as top 2


Results Example

In [ ]:
dd_model = T5Model("t5-base")
dd_model.model.resize_token_embeddings(32128)
checkpoint = torch.load("/content/drive/MyDrive/NLP project/Option3-trained-on-daily-dialog-splitted-corpus-second-epoch.ckpt")
dd_model.load_state_dict(checkpoint['state_dict'])
dd_model.to(DEVICE)

trained_model = T5Model(MODEL_NAME)
trained_model.load_from_checkpoint_with_custom_tokenizer(tokenizer, "/content/drive/MyDrive/style_transfer_model-9-epochs-on-new-data-and-more.ckpt")
trained_model.to(DEVICE)

In [28]:
context_manager = DialogContextManager(dd_model, tokenizer, i_max_token_len, DEVICE ,trained_model)
print(context_manager.write("Where do you work?"))

Question: Where do you work?
Answer: I am the regional manager of Dunder-Mifflin, Michael Scott


In [27]:
context_manager = DialogContextManager(dd_model, tokenizer, i_max_token_len, DEVICE ,trained_model)
print(context_manager.write("Who is your best friend?"))

Question: Who is your best friend?
Answer: My mom, Pam's mom


In [38]:
context_manager = DialogContextManager(dd_model, tokenizer, i_max_token_len, DEVICE ,trained_model)
print(context_manager.write("Who is your best boss?", num_beams=5))

Question: Who is your best boss?
Answer: My boss is coming in today, the lovely Jan Levinson-Gould will, well, no Goulfd
